# 55 Public, Cleveland Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each scenario separately. These files will be used in the Jupyter Notebook.

3. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion

4. Rerun batch sims in EnergyPlus v9.5.
5. Add scenario names, elevations, and glazing systems to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
resgas_cost = 1.16
reselec_cost = 0.1238

comgas_cost = 0.56
comelec_cost = 0.0972

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.93

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI

sf = 329.55

In [6]:
# Lighting energy reduction, assumption that LEDs are being used.

lighting_redux = .3

---

## Combine CSV Files in Directory Commercial Electric Heating

In [7]:
# Create variable for files in directory
files = [f for f in os.listdir("data/cle/ResElecElec/") if f.endswith(".csv")]

# files

In [8]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ResElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/cle/ResElecElec/" + file)
    all_ResElec = pd.concat([all_ResElec, df])
    
all_ResElec

# 105120  rows, 108 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE PEOPLE 1:People Occupant Count [](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly),COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly),EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly),...,COMFENZONE PEOPLE 1:Zone Thermal Comfort Fanger Model PPD [%](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met Time [hr](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System NaturalGas Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly)
0,cle_res_elec_10,E,Exist_LowE,01/01 01:00:00,-1.2000,1.500,15904.72675,0.0,0.0,0.0,...,16.045265,-1.0,-1.0,-0.886806,-1.0,0.0,3565848.557,0,7.151005e+05,0.0
1,cle_res_elec_10,E,Exist_LowE,01/01 02:00:00,-0.6000,1.500,15904.72675,0.0,0.0,0.0,...,15.885338,-1.0,-1.0,-0.886806,-1.0,0.0,3506993.759,0,7.151005e+05,0.0
2,cle_res_elec_10,E,Exist_LowE,01/01 03:00:00,-0.6000,1.500,15904.72675,0.0,0.0,0.0,...,15.917425,-1.0,-1.0,-0.886806,-1.0,0.0,3588966.686,0,7.151005e+05,0.0
3,cle_res_elec_10,E,Exist_LowE,01/01 04:00:00,-0.6000,1.500,15904.72675,0.0,0.0,0.0,...,15.916149,-1.0,-1.0,-0.886806,-1.0,0.0,3633011.978,0,7.151005e+05,0.0
4,cle_res_elec_10,E,Exist_LowE,01/01 05:00:00,-0.6000,1.500,44390.80451,0.0,0.0,0.0,...,15.914225,-1.0,-1.0,-0.886806,-1.0,0.0,3653283.528,0,7.151005e+05,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,cle_res_elec_09,N,Exist_LowE,12/31 20:00:00,3.4875,1.305,234060.60560,0.0,0.0,0.0,...,15.034123,-1.0,-1.0,-0.714479,-1.0,0.0,1292215.296,0,1.340602e+06,0.0
8756,cle_res_elec_09,N,Exist_LowE,12/31 21:00:00,3.5250,1.305,237383.98130,0.0,0.0,0.0,...,14.801551,-1.0,-1.0,-0.714479,-1.0,0.0,1336613.604,0,1.340602e+06,0.0
8757,cle_res_elec_09,N,Exist_LowE,12/31 22:00:00,3.6750,1.500,164269.71510,0.0,0.0,0.0,...,14.603887,-1.0,-1.0,-0.714479,-1.0,0.0,1348046.103,0,1.340602e+06,0.0
8758,cle_res_elec_09,N,Exist_LowE,12/31 23:00:00,4.2125,1.500,91155.44883,0.0,0.0,0.0,...,14.413026,-1.0,-1.0,-0.714479,-1.0,0.0,1412030.685,0,1.340602e+06,0.0


In [9]:
# Get wanted columns
all_ResElec1 = all_ResElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

#all_ComElec1

In [10]:
all_ResElec1 = all_ResElec1.dropna().reset_index(drop=True)

#all_ResElec1

-----

## Get the Wanted Data (columns)

In [11]:
# Rename Columns
all_ResElec2 = all_ResElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ResElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [12]:
# Copy dataframe to modify and leave original df intact
get_annual_ResElec = all_ResElec2.copy(deep=True)

#get_annual_ResElec

In [13]:
# Convert Joules to KBtu in dataframe
get_annual_ResElec["FanEnergy(kBtu)"] = get_annual_ResElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["HeatingElecEnergy(kBtu)"] = get_annual_ResElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["HeatingGasEnergy(kBtu)"] = get_annual_ResElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["CoolingEnergy(kBtu)"] = get_annual_ResElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ResElec["LightingEnergy(kBtu)"] = (get_annual_ResElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux


#get_annual_ComElec

In [14]:
# Drop columns
get_annual_ResElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ResElec

In [15]:
# Add energy totals
totalResEnergy = get_annual_ResElec["FanEnergy(kBtu)"] + get_annual_ResElec["HeatingElecEnergy(kBtu)"] + get_annual_ResElec["LightingEnergy(kBtu)"] + get_annual_ResElec["CoolingEnergy(kBtu)"]
        
get_annual_ResElec["AnnualEnergy(kBtu)"] = totalResEnergy

# get_annual_ResElec

-----

## Get Energy per sf 
Add EUI (site) column<br>

In [16]:
# Get data per sf (energy/sf)
get_annual_ResElec["FanEnergy(kBtu/sf)"] = (get_annual_ResElec["FanEnergy(kBtu)"]) / sf
get_annual_ResElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ResElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ResElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ResElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ResElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ResElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ResElec["LightingEnergy(kBtu/sf)"] = (get_annual_ResElec["LightingEnergy(kBtu)"]) / sf

In [17]:
# Establish EUI (energy/sf)
get_annual_ResElec["EUI(kBtu/sf)"] = (get_annual_ResElec["FanEnergy(kBtu)"] +
                                    get_annual_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ResElec["LightingEnergy(kBtu)"] +
                                    get_annual_ResElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ResElec

-----

## Get Carbon per sf 
Add carbon/sf column

In [18]:
# Do carbon calculation using variable (lb/sf)
get_annual_ResElec["CO2_Electric(lb/sf)"] = ((get_annual_ResElec["FanEnergy(kBtu)"] +
                                    get_annual_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ResElec["LightingEnergy(kBtu)"] +
                                    get_annual_ResElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
#get_annual_ResElec

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [19]:
# Get annual costs
get_annual_costs_ResElec = all_ResElec2.copy(deep=True)

In [20]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ResElec["FanCost($)"] = (get_annual_costs_ResElec["FanEnergy[J](Hourly)"] * .000000277777778) * reselec_cost
get_annual_costs_ResElec["HeatingElecCost($)"] = ((get_annual_costs_ResElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * resgas_cost
get_annual_costs_ResElec["HeatingGasCost($)"] = ((get_annual_costs_ResElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * resgas_cost
get_annual_costs_ResElec["CoolingCost($)"] = (get_annual_costs_ResElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * reselec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ResElec["LightingCost($)"] = (get_annual_costs_ResElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [21]:
# Drop columns
get_annual_costs_ResElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

# get_annual_costs_ResElec

In [22]:
# Add cost totals
totalResCost = get_annual_costs_ResElec["FanCost($)"] + get_annual_costs_ResElec["HeatingElecCost($)"] + get_annual_costs_ResElec["HeatingGasCost($)"]  + get_annual_costs_ResElec["CoolingCost($)"] + get_annual_costs_ResElec["LightingCost($)"]
        
get_annual_costs_ResElec["AnnualCost($)"] = totalResCost

# get_annual_costs_ResElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>

In [23]:
# Get data per sf ($/sf)
get_annual_costs_ResElec["FanCost($/sf)"] = (get_annual_costs_ResElec["FanCost($)"]) / sf
get_annual_costs_ResElec["HeatingElecCost($/sf)"] = (get_annual_costs_ResElec["HeatingElecCost($)"]) / sf
get_annual_costs_ResElec["HeatingGasCost($/sf)"] = (get_annual_costs_ResElec["HeatingGasCost($)"]) / sf
get_annual_costs_ResElec["CoolingCost($/sf)"] = (get_annual_costs_ResElec["CoolingCost($)"]) / sf
get_annual_costs_ResElec["LightingCost($/sf)"] = (get_annual_costs_ResElec["LightingCost($)"]) / sf

# get_annual_costs_ResElec

In [24]:
# Establish costs ($/sf)
get_annual_costs_ResElec["AnnualCost($/sf)"] = (get_annual_costs_ResElec["FanCost($)"] +
                                    get_annual_costs_ResElec["HeatingElecCost($)"] +
                                    get_annual_costs_ResElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ResElec["CoolingCost($)"] +  
                                    get_annual_costs_ResElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ResElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [25]:
# Combine data for energy and cost
allAnnualDataResElec = pd.merge(get_annual_ResElec, get_annual_costs_ResElec)

#allAnnualDataResElec

In [26]:
# list(allAnnualDataResElec.columns.values)

---

## Establish Annual Baseline at Existing

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [27]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnualDataResElec = allAnnualDataResElec[allAnnualDataResElec["GlazingSystem"] == "Exist"]

#baselineAnnualDataResElec

In [28]:
# Rename columns to indicate baseline to prepare for merge\n"

allBaselineAnnualDataResElec = baselineAnnualDataResElec.rename(columns={"Scenario":"Scenario",
 "Date_Time":"Date_Time",
 "Elevation":"Elevation",
 "GlazingSystem":"B_GlazingSystem",
 "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
 "HeatingElecEnergy(kBtu)":"B_HeatingElecEnergy(kBtu)",
 "HeatingGasEnergy(kBtu)":"B_HeatingGasEnergy(kBtu)",
 "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
 "LightingEnergy(kBtu)":"B_LightingEnergy(kBtu)",
 "AnnualEnergy(kBtu)":"B_AnnualEnergy(kBtu)",
 "FanEnergy(kBtu/sf)":"B_FanEnergy(kBtu/sf)",
 "HeatingElecEnergy(kBtu/sf)":"B_HeatingElecEnergy(kBtu/sf)",
 "HeatingGasEnergy(kBtu/sf)":"B_HeatingGasEnergy(kBtu/sf)",
 "CoolingEnergy(kBtu/sf)":"B_CoolingEnergy(kBtu/sf)",
 "LightingEnergy(kBtu/sf)":"B_LightingEnergy(kBtu/sf)",
 "EUI(kBtu/sf)":"B_EUI(kBtu/sf)",
 "CO2_Electric(lb/sf)":"B_CO2_Electric(lb/sf)",
 "FanCost($)":"B_FanCost($)",
 "HeatingElecCost($)":"B_HeatingElecCost($)",
 "HeatingGasCost($)":"B_HeatingGasCost($)",
 "CoolingCost($)":"B_CoolingCost($)",
 "LightingCost($)":"B_LightingCost($)",
 "AnnualCost($)":"B_AnnualCost($)",
 "FanCost($/sf)":"B_FanCost($/sf)",
 "HeatingElecCost($/sf)":"B_HeatingElecCost($/sf)",
 "HeatingGasCost($/sf)":"B_HeatingGasCost($/sf)",
 "CoolingCost($/sf)":"B_CoolingCost($/sf)",
 "LightingCost($/sf)":"B_LightingCost($/sf)",
 "AnnualCost($/sf)":"B_AnnualCost($/sf)"
                            })
#allBaselineAnnualDataResElec

In [29]:
# Merge the dataframes to include the baseline data.
AnnualWithBaselineRes = pd.merge(allAnnualDataResElec, allBaselineAnnualDataResElec, on=["Date_Time", "Elevation", "Scenario"], how="inner")

#AnnualWithBaselineRes

In [30]:
#list(AnnualWithBaseline.columns.values)

In [31]:
# Add column for the difference from baseline columns (scenario - baseline)
AnnualWithBaselineRes["deltaEnergy(kBtu)"] = AnnualWithBaselineRes["AnnualEnergy(kBtu)"] - AnnualWithBaselineRes["B_AnnualEnergy(kBtu)"]
AnnualWithBaselineRes["deltaEUI(kBtu/sf)"] = AnnualWithBaselineRes["EUI(kBtu/sf)"] - AnnualWithBaselineRes["B_EUI(kBtu/sf)"]
AnnualWithBaselineRes["deltaCO2Elec(lb/sf)"] = AnnualWithBaselineRes["CO2_Electric(lb/sf)"] - AnnualWithBaselineRes["B_CO2_Electric(lb/sf)"]
AnnualWithBaselineRes["deltaCost($)"] = AnnualWithBaselineRes["AnnualCost($)"] - AnnualWithBaselineRes["B_AnnualCost($)"]
AnnualWithBaselineRes["deltaCost($/sf)"] = AnnualWithBaselineRes["AnnualCost($/sf)"] - AnnualWithBaselineRes["B_AnnualCost($/sf)"]

# AnnualWithBaseline

In [32]:
# Convert NANs to 0
AnnualWithBaselineRes =AnnualWithBaselineRes.fillna(0)

#AnnualWithBaseline

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [33]:
# Group by scenario
annualDataRes = AnnualWithBaselineRes.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

#annualData.sum().round(2)

In [34]:
annualDataResFinal = annualDataRes.sum().round(2)

#annualDataResFinal

In [35]:
# Export grouped file to csv\n
annualDataResFinal.to_csv("Scraper_Output/cle_annual_res.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [36]:
# Start with renaming df
get_peak_ResElec = get_annual_ResElec
#get_peak_ComElec

In [37]:
# Add up heat, cool and fans for a total column
get_peak_ResElec["TotalEnergy(kBtu)"] = (get_peak_ResElec["FanEnergy(kBtu)"] +
                                    get_peak_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_peak_ResElec["LightingEnergy(kBtu)"] +
                                    get_peak_ResElec["CoolingEnergy(kBtu)"])

#get_peak_ComElec

In [38]:
# Get scenario and find the max
scenario1 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_01"]
scenario2 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_02"]
scenario3 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_03"]
scenario4 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_04"]
scenario5 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_05"]
scenario6 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_06"]
scenario7 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_07"]
scenario8 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_08"]
scenario9 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_09"]
scenario10 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_10"]
scenario11 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_11"]
scenario12 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_12"]

In [39]:
scenarioPeak1 = (scenario1[scenario1["TotalEnergy(kBtu)"] == scenario1["TotalEnergy(kBtu)"].max()])
scenarioPeak2 = (scenario2[scenario2["TotalEnergy(kBtu)"] == scenario2["TotalEnergy(kBtu)"].max()])
scenarioPeak3 = (scenario3[scenario3["TotalEnergy(kBtu)"] == scenario3["TotalEnergy(kBtu)"].max()])
scenarioPeak4 = (scenario4[scenario4["TotalEnergy(kBtu)"] == scenario4["TotalEnergy(kBtu)"].max()])
scenarioPeak5 = (scenario5[scenario5["TotalEnergy(kBtu)"] == scenario5["TotalEnergy(kBtu)"].max()])
scenarioPeak6 = (scenario6[scenario6["TotalEnergy(kBtu)"] == scenario6["TotalEnergy(kBtu)"].max()])
scenarioPeak7 = (scenario7[scenario7["TotalEnergy(kBtu)"] == scenario7["TotalEnergy(kBtu)"].max()])
scenarioPeak8 = (scenario8[scenario8["TotalEnergy(kBtu)"] == scenario8["TotalEnergy(kBtu)"].max()])
scenarioPeak9 = (scenario9[scenario9["TotalEnergy(kBtu)"] == scenario9["TotalEnergy(kBtu)"].max()])
scenarioPeak10 = (scenario10[scenario10["TotalEnergy(kBtu)"] == scenario10["TotalEnergy(kBtu)"].max()])
scenarioPeak11 = (scenario11[scenario11["TotalEnergy(kBtu)"] == scenario11["TotalEnergy(kBtu)"].max()])
scenarioPeak12 = (scenario12[scenario12["TotalEnergy(kBtu)"] == scenario12["TotalEnergy(kBtu)"].max()])

In [40]:
PeakDemandRes =scenarioPeak1.append([scenarioPeak2, 
                                  scenarioPeak3,
                                  scenarioPeak4,
                                  scenarioPeak5,
                                  scenarioPeak6,
                                  scenarioPeak7,
                                  scenarioPeak8,
                                  scenarioPeak9,
                                  scenarioPeak10,
                                  scenarioPeak11,
                                  scenarioPeak12])

PeakDemandRes

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf),TotalEnergy(kBtu)
27149,cle_res_elec_01,02/06 06:00:00,N,Exist,1.811737,11.603646,0.0,0.000000,0.026595,13.441977,0.005498,0.035211,0.0,0.000000,0.000081,0.040789,0.017173,13.441977
35909,cle_res_elec_02,02/06 06:00:00,E,Exist,0.942901,12.512882,0.0,0.000000,0.026595,13.482378,0.002861,0.037970,0.0,0.000000,0.000081,0.040911,0.017225,13.482378
44669,cle_res_elec_03,02/06 06:00:00,S,Exist,1.635596,11.637495,0.0,0.000000,0.026595,13.299686,0.004963,0.035313,0.0,0.000000,0.000081,0.040357,0.016992,13.299686
53429,cle_res_elec_04,02/06 06:00:00,W,Exist,1.462425,11.602939,0.0,0.000000,0.026595,13.091959,0.004438,0.035208,0.0,0.000000,0.000081,0.039727,0.016726,13.091959
66113,cle_res_elec_05,07/19 18:00:00,N,Exist_Clear,1.620319,0.000000,0.0,8.419022,0.041579,10.080920,0.004917,0.000000,0.0,0.025547,0.000126,0.030590,0.012879,10.080920
70949,cle_res_elec_06,02/06 06:00:00,E,Exist_Clear,0.870359,7.780903,0.0,0.000000,0.026595,8.677856,0.002641,0.023611,0.0,0.000000,0.000081,0.026332,0.011087,8.677856
83626,cle_res_elec_07,07/19 11:00:00,S,Exist_Clear,1.439699,0.000000,0.0,7.476732,0.008032,8.924463,0.004369,0.000000,0.0,0.022688,0.000024,0.027081,0.011402,8.924463
93040,cle_res_elec_08,08/15 17:00:00,W,Exist_Clear,1.271018,0.000000,0.0,7.017282,0.029632,8.317931,0.003857,0.000000,0.0,0.021294,0.000090,0.025240,0.010627,8.317931
97229,cle_res_elec_09,02/06 06:00:00,N,Exist_LowE,1.270645,7.604719,0.0,0.000000,0.026595,8.901959,0.003856,0.023076,0.0,0.000000,0.000081,0.027012,0.011373,8.901959
869,cle_res_elec_10,02/06 06:00:00,E,Exist_LowE,0.677785,8.227031,0.0,0.000000,0.026595,8.931411,0.002057,0.024964,0.0,0.000000,0.000081,0.027102,0.011411,8.931411


In [41]:
# Export dataframe  to csv
PeakDemandRes.to_csv("Scraper_Output/cle_peak_res.csv", header=True, index=True)